In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("NaiveBayes")\
        .getOrCreate()

In [3]:
dataset = spark.read.csv("diabetes.csv",header=True)

In [4]:
dataset.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [5]:
dataset.printSchema()

root
 |-- Pregnancies: string (nullable = true)
 |-- Glucose: string (nullable = true)
 |-- BloodPressure: string (nullable = true)
 |-- SkinThickness: string (nullable = true)
 |-- Insulin: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- DiabetesPedigreeFunction: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Outcome: string (nullable = true)



In [6]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [7]:
new_data.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Outcome: float (nullable = true)



In [8]:
from pyspark.sql.functions import col, count, isnan, when
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [9]:
cols=new_data.columns
cols.remove("Outcome")
assembler = VectorAssembler(inputCols=cols,outputCol="features")
# Now let us use the transform method to transform our dataset
data=assembler.transform(new_data)
data = data.select("features",'Outcome')

In [10]:
(train, test) = data.randomSplit([0.7, 0.3])

In [11]:
naive_bayes = NaiveBayes(featuresCol='features',labelCol='Outcome',smoothing=1.0)

In [12]:
model = naive_bayes.fit(train) 

In [13]:
# select example rows to display.
predictions = model.transform(test)

In [14]:
predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[1,5,6,7],[117...|    0.0|[-323.61224216275...|[0.09454563319014...|       1.0|
|(8,[1,5,6,7],[138...|    1.0|[-307.81086275721...|[0.01681026466627...|       1.0|
|(8,[1,5,6,7],[145...|    1.0|[-346.93881460780...|[0.02407280353738...|       1.0|
|[0.0,67.0,76.0,0....|    0.0|[-417.06677625264...|[0.99999931459960...|       0.0|
|[0.0,74.0,52.0,10...|    0.0|[-370.58046002135...|[0.99276890812465...|       0.0|
|[0.0,84.0,64.0,22...|    0.0|[-499.12358955402...|[0.99356710387183...|       0.0|
|[0.0,84.0,82.0,31...|    0.0|[-651.79943709385...|[0.93393807821312...|       0.0|
|[0.0,91.0,80.0,0....|    0.0|[-376.62699782289...|[0.99999309663815...|       0.0|
|[0.0,93.0,60.0,25...|    0.0|[-536.09847424452...|[0.69234538911861...|    

In [15]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)


In [16]:
print ("Accuracy",accuracy)

Accuracy 0.6697674418604651


In [17]:
spark.stop()